<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

In [ ]:
from esper.prelude import *
from esper.widget import *
from esper.spark_util import *
from esper.major_canonical_shows import MAJOR_CANONICAL_SHOWS

import math

In [ ]:
phrase = 'make america great again'

In [ ]:
caption_matches = caption_search([phrase.upper()])[0]

In [ ]:
face_genders = get_face_genders()
face_genders = annotate_interval_overlap(
    face_genders.where(face_genders.in_commercial == False),
    caption_matches
)
face_genders = face_genders.where(face_genders.overlap_seconds > 0.)

In [ ]:
z_score = 1.96
male_face_count, male_face_var = count_distinct_over_column(
    face_genders, ['face_id'], probability_column='male_probability'
)
print('Male face count: {:0.2f} +/- {:0.2f}'.format(
    male_face_count, z_score * math.sqrt(male_face_var)))
female_face_count, female_face_var = count_distinct_over_column(
    face_genders, ['face_id'], probability_column='female_probability'
)
print('Female face count: {:0.2f} +/- {:0.2f}'.format(
    female_face_count, z_score * math.sqrt(female_face_var)))

male_nh_face_count, male_nh_face_var = count_distinct_over_column(
    face_genders.where(face_genders.host_probability < 0.25),
    ['face_id'], probability_column='male_probability'
)
print('Male (non-host) face count: {:0.2f} +/- {:0.2f}'.format(
    male_nh_face_count, z_score * math.sqrt(male_nh_face_var)))
female_nh_face_count, female_nh_face_var = count_distinct_over_column(
    face_genders.where(face_genders.host_probability < 0.25), 
    ['face_id'], probability_column='female_probability'
)
print('Female (non-host) face count: {:0.2f} +/- {:0.2f}'.format(
    female_nh_face_count, z_score * math.sqrt(female_nh_face_var)))

In [ ]:
esper_widget(
    qs_to_result(
        FaceGender.objects.filter(
            id__in=[
                x.id for x in face_genders.where(
                    (face_genders.host_probability < 0.25) &
                    (face_genders.size_percentile > 25)
                ).select('id').collect()
            ]
        ),
        shuffle=True,
        limit=10000
    ),
    crop_bboxes=True
)

In [ ]:
clusters = defaultdict(list)
for (i, c) in face_kmeans([
            x.face_id for x in face_genders.where(
                (face_genders.host_probability < 0.25) &
                (face_genders.size_percentile > 25) &
                (face_genders.blurriness > 10)
            ).select('face_id').collect()
        ], k=100):
    clusters[c].append(i)

kmeans_results = []
for _, ids in sorted(clusters.items(), key=lambda x: -len(x[1])):
    kmeans_results.append((
        'Cluster with {} faces'.format(len(ids)), 
        qs_to_result(Face.objects.filter(id__in=ids).distinct('shot__video'), 
                     limit=10)
    ))

esper_widget(group_results(kmeans_results), crop_bboxes=True)